In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Initialization of Variables
P_thruster = 1250000 #newtons
P_lift = 1400000 #newtons
P_weight = 500000 #newtons

alpha = 5 #degrees
alpha = alpha * np.pi/180

# Equations for P1 & P2 (Horizontal and Vertical Resultant Forces)
P1 = P_lift*np.sin(alpha) - P_thruster

def getT(beta):
    beta = beta * np.pi/180
    T = np.array([[np.cos(beta)**2, np.sin(beta)**2, 2*np.sin(beta)*np.cos(beta)],
                 [np.sin(beta)**2, np.cos(beta)**2,-2*np.sin(beta)*np.cos(beta)],
                 [-1*np.sin(beta)*np.cos(beta), np.sin(beta)*np.cos(beta),np.cos(beta)**2 - np.sin(beta)**2]])
    return T

print(getT(90))

[[ 3.74939946e-33  1.00000000e+00  1.22464680e-16]
 [ 1.00000000e+00  3.74939946e-33 -1.22464680e-16]
 [-6.12323400e-17  6.12323400e-17 -1.00000000e+00]]
